<br>

# Introdução

In [ ]:
import os
import re
import time
import json
import folium
import random
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from folium import plugins
from osgeo import gdal, osr
from bs4 import BeautifulSoup
from tqdm.notebook import trange, tqdm

<br>

# Join Data

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp_policiaambiental/main/data/tabs/tab_municipio_cpamb.csv',
)

# Lê o arquivo csv com o nome dos municípios
gdf = gpd.read_file(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/shps/sp_250k_wgs84.geojson',
)
gdf.drop(['municipio_nome'], axis=1, inplace=True)
gdf['id_municipio'] = gdf['id_municipio'].astype(int)
gdf['geometry'] = gdf.simplify(0.0015)

# Merge
gdf = gdf.merge(
    df,
    on='id_municipio',
    how='left'
)

# Results
gdf.head()

In [ ]:
# Batalhão
gdf['id_batalhao_desc'] = gdf['id_batalhao'].apply(lambda x: '{}º Batalhão'.format(x))

# Companhia
gdf['id_cia_desc'] = gdf['id_cia'].apply(lambda x: '{}ª Companhia do '.format(x))
gdf['id_cia_desc'] = gdf[['id_cia_desc', 'id_batalhao_desc']].apply(lambda x: ''.join(x), axis=1)

# Pelotão
gdf['id_pelotao_desc'] = gdf['id_pelotao'].apply(lambda x: '{}º Pelotão da '.format(x))
gdf['id_pelotao_desc'] = gdf[['id_pelotao_desc', 'id_cia_desc']].apply(lambda x: ''.join(x), axis=1)

# Results
gdf.head()

In [ ]:
set(gdf['id_cia_desc'])

<br>

## Adjust Table

In [ ]:
# Save geojson
gdf.to_file(
    os.path.join('data', 'shps', 'sp_cpamb.geojson'),
    driver='GeoJSON',
    encoding='utf-8'
)

# Results
gdf.head()

<br>

## Layers

In [ ]:
def add_lyr_cartodbpositron(min_zoom, max_zoom):
    lyr = folium.TileLayer(        
        tiles='cartodbpositron',
        name='CartoDB Positron',
        min_zoom=min_zoom,
        max_zoom=max_zoom,        
        overlay=False,
        control=False,
        show=False,
    )
    return lyr

In [ ]:
def add_lyr_batalhao():
    # Input
    gdf = gpd.read_file(os.path.join('data', 'shps', 'sp_cpamb.geojson'))
    gdf = gdf.to_crs(epsg=4326)
    
    # Column with category
    col_categories1 = 'id_batalhao_desc'
    
    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories1])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))    
    color_polygon['1º Batalhão'] = '#ff4d4d'
    #color_polygon['2º Batalhão'] = '#4da6ff'
    color_polygon['3º Batalhão'] = '#00b300'
    color_polygon['4º Batalhão'] = '#ffcc99'
    color_polygon1 = color_polygon
    
    # Popup
    gdf['popup_batalhao'] = gdf.apply(popup_html_batalhao, axis=1)
    
    # Geo
    lyr = folium.GeoJson(
        gdf,
        name='Batalhões',
        smooth_factor=1.0,
        zoom_on_click=False,
        embed=False,
        show=False,
        style_function=lambda x: {
            'fillColor': color_polygon1[x['properties'][col_categories1]],
            'color':     color_polygon1[x['properties'][col_categories1]],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'id_batalhao_desc'],
            aliases=['Munícipio', 'Batalhão'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup_batalhao'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,            
        )
    )    
    return lyr

# Add Field
def popup_html_batalhao(row):
    #tel = str(row['telefone']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    #fax = str(row['fax']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    
    html = """
    <div>
    <p><b>{}</b> pertence ao:
    <h5><b>{}</b></h5></p>
    </div>
    """.format(
        '' if pd.isnull(row['municipio_nome']) else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['id_batalhao_desc'])        else '{}'.format(row['id_batalhao_desc']),
    )
    
    html = html.replace('\n','')
    html = re.sub('\s\s+' , ' ', html) # Remove Espaços no meio
    html = html.strip()
    return html

In [ ]:
def add_lyr_cpamb_companhia():
    # Input
    gdf = gpd.read_file(os.path.join('data', 'shps', 'sp_cpamb.geojson'))
    gdf = gdf.to_crs(epsg=4326)
    
    # Column with category
    col_categories2 = 'id_cia_desc'
    
    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories2])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))
    color_polygon2 = color_polygon

    # Popup
    gdf['popup_cia'] = gdf.apply(popup_html_cia, axis=1)
    
    # Geo
    lyr = folium.GeoJson(
        gdf,
        name='Companhias',
        smooth_factor=1.0,
        zoom_on_click=False,
        embed=False,
        show=False,
        style_function=lambda x: {
            'fillColor': color_polygon2[x['properties'][col_categories2]],
            'color':     color_polygon2[x['properties'][col_categories2]],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'id_cia_desc'],
            aliases=['Munícipio', 'Companhia'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup_cia'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,            
        )
    )
    return lyr

# Add Field
def popup_html_cia(row):
    #tel = str(row['telefone']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    #fax = str(row['fax']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    
    html = """
    <div>
    <p><b>{}</b> pertence ao:
    <h5><b>{}</b></h5></p>
    </div>
    """.format(
        '' if pd.isnull(row['municipio_nome']) else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['id_cia_desc'])        else '{}'.format(row['id_cia_desc']),
    )
    
    html = html.replace('\n','')
    html = re.sub('\s\s+' , ' ', html) # Remove Espaços no meio
    html = html.strip()
    return html

In [ ]:
def add_lyr_cpamb_pelotao():
    # Input
    gdf = gpd.read_file(os.path.join('data', 'shps', 'sp_cpamb.geojson'))
    gdf = gdf.to_crs(epsg=4326)
    
    # Column with category
    col_categories3 = 'id_pelotao_desc'
    
    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories3])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))
    color_polygon3 = color_polygon
    
    # Popup
    gdf['popup_pelotao'] = gdf.apply(popup_html_pelotao, axis=1)
    
    # Geo
    lyr = folium.GeoJson(
        gdf,
        name='Pelotão',
        smooth_factor=1.0,
        zoom_on_click=False,
        embed=False,
        show=True,
        style_function=lambda x: {
            'fillColor': color_polygon3[x['properties'][col_categories3]],
            'color':     color_polygon3[x['properties'][col_categories3]],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'id_pelotao_desc'],
            aliases=['Munícipio', 'Pelotão'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup_pelotao'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,            
        )
    )
    return lyr

# Add Field
def popup_html_pelotao(row):
    #tel = str(row['telefone']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    #fax = str(row['fax']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    
    html = """
    <div>
    <p><b>{}</b> pertence ao:
    <h5><b>{}</b></h5></p>
    </div>
    """.format(
        '' if pd.isnull(row['municipio_nome'])  else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['id_pelotao_desc']) else '{}'.format(row['id_pelotao_desc']),
    )
    
    html = html.replace('\n','')
    html = re.sub('\s\s+' , ' ', html) # Remove Espaços no meio
    html = html.strip()
    return html

<br>

## Folium

In [ ]:
def get_map(input_geojson):
    # Input
    gdf = gpd.read_file(input_geojson)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]
    
    # Zoom
    min_zoom = 6
    max_zoom = 11
    
    # Create Map
    m = folium.Map(
        #zoom_start=6,
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        #zoom_delta=0.1,
        min_lat=bounds[0][0]*(101/100),
        min_lon=bounds[0][1]*(101/100),
        max_lat=bounds[1][0]*(99/100),
        max_lon=bounds[1][1]*(99/100),
        tiles=None,
    )
    
    # Add Layers
    m.add_child(add_lyr_cartodbpositron(min_zoom, max_zoom))
    m.add_child(add_lyr_cpamb_pelotao())
    m.add_child(add_lyr_cpamb_companhia())
    m.add_child(add_lyr_batalhao())    
    
    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False,
    ).add_to(m)
    return m

In [ ]:
# Map without Bounds
m = get_map(os.path.join('data', 'shps', 'sp_cpamb.geojson'))

# Results
os.makedirs('maps', exist_ok=True)
m.save(os.path.join('maps', 'cpamb_map.html'))
#m.save(os.path.join('..', '..', '..', 'case_django', 'divadmin', 'templates', 'cpamb_map.html'))
m